# Stock Market Question Answering Agent from OpenBB JSON


In [20]:
import os
import sys
import dotenv
import warnings
from datetime import datetime, timedelta
import re

import requests
import base64
import json
import yaml

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ChatMessageHistory
from langchain.tools import StructuredTool
from langchain.requests import RequestsWrapper

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.agent_toolkits.openapi import planner
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec

import wikipedia

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

from BB_agent_tool import BB_agent_tool, get_response, agent_query, bb_agent_system_prompt

import pdb

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [7]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')
print(f'wikipedia      {wikipedia.__version__}')


pandas         2.2.2
obb            4.1.7
openai         1.28.0
langchain      0.1.20
wikipedia      (1, 4, 0)


In [8]:
MODEL = "gpt-4o"

# MAX_INPUT_TOKENS = 65536     
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0


In [9]:
# import os
# import json
# import yaml

# # Load the JSON file
# with open('openapi.json', 'r') as json_file:
#     json_data = json.load(json_file)

# # Convert JSON data to YAML
# yaml_data = yaml.dump(json_data, default_flow_style=False, sort_keys=False)

# # Save the YAML data to a file
# with open('openapi.yaml', 'w') as yaml_file:
#     yaml_file.write(yaml_data)

# print("Conversion complete: 'openapi.json' -> 'openapi.yaml'")


In [10]:
with open("openapi.yaml") as f:
    raw_openbb_api_spec = yaml.load(f, Loader=yaml.Loader)
    
openbb_api_spec = reduce_openapi_spec(raw_openbb_api_spec)


In [11]:
endpoints = [
    (route, operation)
    for route, operations in raw_openbb_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]
len(endpoints)


203

In [12]:
enc = tiktoken.encoding_for_model(MODEL)

def count_tokens(s):
    return len(enc.encode(s))

count_tokens(yaml.dump(raw_openbb_api_spec))


383362

# Connect to OpenBB

In [13]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)

In [14]:
obb.equity.search("Merck", provider="nasdaq").to_df().head(3)


,symbol,name,nasdaq_traded,exchange,etf,round_lot_size,test_issue,cqs_symbol,nasdaq_symbol,next_shares
0,MRK,"Merck & Company, Inc. Common Stock (new)",Y,N,N,100.0,N,MRK,MRK,N


In [15]:
symbol = "MRK"
company = "Merck"

In [16]:
obj = obb.equity.price.quote(symbol)
obj


OBBject

id: 0664a4ac-e26a-73ad-8000-1cec61d88c2d
results: [{'symbol': 'MRK', 'asset_type': 'stock', 'name': 'MERCK & CO INC COM', 'e...
provider: cboe
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'cboe'}, 'stand...

In [17]:
# use REST API on server running locally
# uvicorn openbb_core.api.rest_api:app --host 0.0.0.0 --port 8000 --reload
# REST API documentation - http://127.0.0.1:8000/docs
# openapi.json : http://127.0.0.1:8000/openapi.json

# not turning on authentication
# msg = "some_user:some_pass"
# msg_bytes = msg.encode('ascii')
# base64_bytes = base64.b64encode(msg_bytes)
# base64_msg = base64_bytes.decode('ascii')

url = f"http://127.0.0.1:8000/api/v1/equity/price/quote?provider=yfinance&symbol={symbol}"
# headers = {"accept": "application/json", "Authorization": f"Basic {base64_msg}"}
headers = {"accept": "application/json"}

response = requests.get(url=url, headers=headers)

response.json()


{'results': [{'symbol': 'MRK',
   'asset_type': 'EQUITY',
   'name': 'Merck & Co., Inc.',
   'exchange': 'NYQ',
   'bid': 126.0,
   'bid_size': 1000,
   'ask': 133.0,
   'ask_size': 800,
   'last_price': 131.19,
   'open': 130.69,
   'high': 131.32,
   'low': 129.81,
   'volume': 8281595,
   'prev_close': 130.88,
   'year_high': 133.1,
   'year_low': 99.14,
   'ma_50d': 127.1046,
   'ma_200d': 114.8178,
   'volume_average': 8218136.0,
   'volume_average_10d': 6913350.0,
   'currency': 'USD'}],
 'provider': 'yfinance',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {'use_cache': True, 'source': 'iex'}},
   'duration': 146260000,
   'route': '/equity/price/quote',
   'timestamp': '2024-05-19T14:54:11.033079'}}}

# Prompt OpenAI 

In [19]:
llm = ChatOpenAI(model_name=MODEL, temperature=0.0)


In [21]:
requests_wrapper = RequestsWrapper(headers=headers)


In [22]:
# NOTE: set allow_dangerous_requests manually for security concern https://python.langchain.com/docs/security
ALLOW_DANGEROUS_REQUEST=True

openbb_agent = planner.create_openapi_agent(
    openbb_api_spec,
    requests_wrapper,
    llm,
    allow_dangerous_requests=ALLOW_DANGEROUS_REQUEST,
)
user_query = "get the latest quote for symbol NVDA."

openbb_agent.invoke(user_query)




> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API call to get the latest quote for the symbol NVDA.
Observation: 1. **Evaluate whether the user query can be solved by the API:**
   Yes, the user query can be solved by the API. The user wants to get the latest quote for the symbol NVDA, which is a stock symbol. The API provides an endpoint to get the latest quote for a given stock.

2. **Generate a plan of API calls and say what they are doing step by step:**
   - **Step 1:** Use the endpoint `GET /api/v1/equity/price/quote` to get the latest quote for the stock symbol NVDA.

**Plan:**
1. **GET /api/v1/equity/price/quote**
   - **Description:** This API call will retrieve the latest quote for the stock symbol NVDA, including price, volume, and other relevant data.
   - **Parameters:** 
     - `symbol`: NVDA

This single API call will provide the necessary information to fulfill the user's query.
Thought:I have the plan to get the late

KeyboardInterrupt: 